In [1]:
import pandas as pd
from result_df import find_anomaly_problem_chain
from log_to_df import logs_to_dataframe, collect_all_testcases
from chain_with_transformer import find_anomaly_problem_chain_transform


D:\AtomicHack-Log-Monitor\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загрузка датасета аномалий и проблем
anomalies_problems = pd.read_csv('../Validation_Cases/ValidationCase_1/anomalies_problems.csv', sep=';')

In [3]:
anomalies_problems.head()

,ID аномалии,Аномалия,ID проблемы,Проблема
0,1,ata1: COMRESET failed,1,RAID controller firmware fault detected
1,2,RAID array degraded,1,RAID controller firmware fault detected
2,3,RAID disk offline,1,RAID controller firmware fault detected
3,4,ata1: read timeout,1,RAID controller firmware fault detected
4,5,Disk status unknown,1,RAID controller firmware fault detected


In [4]:
anomalies_problems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID аномалии  500 non-null    int64 
 1   Аномалия     500 non-null    object
 2   ID проблемы  500 non-null    int64 
 3   Проблема     500 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.8+ KB


In [5]:
print(f'Количество уникальных значений аномалий text: {len(anomalies_problems['Аномалия'].unique())}')
print(f'Количество уникальных значений аномалий id: {len(anomalies_problems['ID аномалии'].unique())}')
print(f'Количество уникальных значений проблем text: {len(anomalies_problems['Проблема'].unique())}')
print(f'Количество уникальных значений проблем id: {len(anomalies_problems['ID проблемы'].unique())}')

Количество уникальных значений аномалий text: 399
Количество уникальных значений аномалий id: 410
Количество уникальных значений проблем text: 50
Количество уникальных значений проблем id: 50


In [6]:
# Проверка на дубликаты
all_duplicates = anomalies_problems.duplicated(keep=False)
duplicate_rows = anomalies_problems[all_duplicates]
len(duplicate_rows)

0

In [7]:
# Количество аномалий к каждой проблеме
ap_group_by_problem = anomalies_problems.groupby('ID проблемы').agg({'ID аномалии': 'count'})
ap_group_by_problem

,ID аномалии
ID проблемы,
1,10
2,10
3,10
4,10
5,10
6,10
7,10
8,10
9,10


In [8]:
# Количество проблем к каждой аномалии
ap_group_by_anomalies = anomalies_problems.groupby('ID аномалии').agg({'ID проблемы': 'count'})
ap_group_by_anomalies[ap_group_by_anomalies['ID проблемы']>1]

,ID проблемы
ID аномалии,
481,10
482,10
483,10
484,10
485,10
486,10
487,10
488,10
489,10


In [9]:
anomalies_problems[anomalies_problems['ID аномалии']==481]

,ID аномалии,Аномалия,ID проблемы,Проблема
8,481,segfault at 0x0 ip 0x7f1234567890,1,RAID controller firmware fault detected
58,481,segfault at 0x0 ip 0x7f1234567890,6,Motherboard VRM failure detected
108,481,segfault at 0x0 ip 0x7f1234567890,11,Syntax error in module initialization
158,481,segfault at 0x0 ip 0x7f1234567890,16,Software update installation failed
208,481,segfault at 0x0 ip 0x7f1234567890,21,Network interface eth0 link failure
258,481,segfault at 0x0 ip 0x7f1234567890,26,SSL certificate expired
308,481,segfault at 0x0 ip 0x7f1234567890,31,Database instance stopped unexpectedly
358,481,segfault at 0x0 ip 0x7f1234567890,36,Replication lag exceeded threshold
408,481,segfault at 0x0 ip 0x7f1234567890,41,Data center cooling system failure
458,481,segfault at 0x0 ip 0x7f1234567890,46,Virtual network misconfiguration


**!!!Вопрос что делать с аномалиями, которые относятся к нескольким проблемам!!!**

In [10]:
# Нужные логи кейса 1
VC1 = logs_to_dataframe('../Validation_Cases/ValidationCase_1')
VC1.head()

,datetime,level,source,text,filename,line_number
0,2025-10-03T00:18:00,ERROR,hardware,Network interface eth0 link failure,web_server_log.txt,39601
1,2025-10-03T02:29:31,WARNING,network,segfault at 0x0 ip 0x7f1234567890,web_server_log.txt,47492


In [11]:
# Нужные логи кейса 2
VC2 = logs_to_dataframe('../Validation_Cases/ValidationCase_2')
VC2.head()

,datetime,level,source,text,filename,line_number
0,2025-10-06T01:55:34,WARNING,database,Slow query from index issue,backup_server_log.txt,6935
1,2025-10-06T05:44:04,WARNING,database,Query timeout warning,router_log.txt,20645
2,2025-10-06T05:09:49,WARNING,database,Application crash on index table,storage_system_log.txt,18590
3,2025-10-06T07:42:23,ERROR,database,Index corruption on table,web_server_log.txt,27744


In [12]:
# Нужные логи кейса 3
VC3 = logs_to_dataframe('../Validation_Cases/ValidationCase_3')
VC3.head(10)

,datetime,level,source,text,filename,line_number
0,2025-10-06T00:24:39,WARNING,network,Performance degradation detected,app_server1_log.txt,1480
1,2025-10-06T08:59:32,ERROR,os,Host overheating alert,app_server1_log.txt,32373
2,2025-10-06T08:36:41,WARNING,os,Overheating alert in host,app_server2_log.txt,31002
3,2025-10-06T09:16:28,WARNING,os,Overheating alert in host,app_server2_log.txt,33389
4,2025-10-06T08:51:26,WARNING,network,Slow virtual network performance,backup_server_log.txt,31887
5,2025-10-06T02:25:21,WARNING,network,User-facing VM network issue,firewall_log.txt,8722
6,2025-10-06T11:41:12,ERROR,network,Virtual network misconfiguration,switch1_log.txt,42073


In [13]:
# Выходной датафрейм
result_df = find_anomaly_problem_chain(VC2, anomalies_problems)
result_df.head()


,ID аномалии,ID проблемы,Файл с проблемой,№ строки,Строка из лога
0,267,34,web_server_log.txt,27744,Index corruption on table
1,271,34,web_server_log.txt,27744,Index corruption on table


In [14]:
# # Выходной датафрейм
# result_df_t = find_anomaly_problem_chain_transform(VC2, anomalies_problems)
# result_df_t.head()

In [15]:
all_tests = collect_all_testcases('../Test Cases')

In [16]:
all_tests.head()

,datetime,level,source,text,filename,line_number,id_scenario
0,2025-11-25T20:12:17,ERROR,network,Network interface eth0 link failure,app_server1_log.txt,12438,1
1,2025-11-26T03:32:19,WARNING,network,eth0: link down,app_server1_log.txt,38840,1
2,2025-01-30T13:10:31,WARNING,software,ata2: I/O timeout,app_server1_log.txt,632,10
3,2025-01-30T13:28:30,WARNING,software,ata2: SError: { BadCRC },app_server1_log.txt,1711,10
4,2025-01-30T13:41:11,WARNING,software,HDD response timeout,app_server1_log.txt,2472,10


In [17]:
df = find_anomaly_problem_chain_transform(all_tests, anomalies_problems)

In [18]:
df_clear = df.drop(['Аномалия', 'Уверенность'], axis=1)

In [19]:
df_clear.rename(columns={'id_scenario': 'ID сценария'}, inplace=True)

In [20]:
df_clear.head()

,ID сценария,ID аномалии,ID проблемы,Файл с проблемой,№ строки,Строка из лога
0,1,65,21.0,app_server1_log.txt,12438.0,2025-11-25T20:12:17 ERROR network: Network int...
1,2,205,26.0,app_server1_log.txt,45017.0,2025-02-08T01:50:16 ERROR database: SSL certif...
2,3,340,43.0,app_server1_log.txt,4993.0,2025-08-18T11:53:12 ERROR hardware: ESXi hyper...
3,4,11,2.0,app_server1_log.txt,47747.0,2025-04-30T22:15:46 ERROR hardware: CPU0: Core...
4,5,35,5.0,app_server1_log.txt,43502.0,2025-12-11T02:05:01 ERROR hardware: SSD wear-l...


In [21]:
df_clear.to_csv("result.csv", index=False, encoding="utf-8-sig")

## Работа с None

### Сценарий 10

In [22]:
all_tests[all_tests['id_scenario']==10].head()

,datetime,level,source,text,filename,line_number,id_scenario
2,2025-01-30T13:10:31,WARNING,software,ata2: I/O timeout,app_server1_log.txt,632,10
3,2025-01-30T13:28:30,WARNING,software,ata2: SError: { BadCRC },app_server1_log.txt,1711,10
4,2025-01-30T13:41:11,WARNING,software,HDD response timeout,app_server1_log.txt,2472,10
5,2025-01-30T14:01:51,ERROR,software,HDD mechanical fault (SMART failure),app_server1_log.txt,3712,10


In [23]:
df[df['id_scenario']==10].head()

,id_scenario,ID аномалии,Аномалия,Уверенность,ID проблемы,Файл с проблемой,№ строки,Строка из лога
15,10,56,ata2: I/O timeout,1.0,7.0,app_server1_log.txt,3712.0,2025-01-30T14:01:51 ERROR software: HDD mechan...
16,10,49,ata2: SError: { BadCRC },1.0,7.0,app_server1_log.txt,3712.0,2025-01-30T14:01:51 ERROR software: HDD mechan...
17,10,52,HDD response timeout,1.0,7.0,app_server1_log.txt,3712.0,2025-01-30T14:01:51 ERROR software: HDD mechan...


In [24]:
anomalies_problems[anomalies_problems['ID аномалии'].isin([49,52,56])]

,ID аномалии,Аномалия,ID проблемы,Проблема
60,49,ata2: SError: { BadCRC },7,HDD mechanical fault (SMART failure)
63,52,HDD response timeout,7,HDD mechanical fault (SMART failure)
67,56,ata2: I/O timeout,7,HDD mechanical fault (SMART failure)


### Сценарий 29

In [26]:
all_tests[all_tests['id_scenario']==29].head()

,datetime,level,source,text,filename,line_number,id_scenario
119,2025-01-08T17:22:13,ERROR,network,Traffic dropped due to rule mismatch,firewall_log.txt,30134,29
120,2025-01-08T22:47:44,WARNING,network,Network anomaly detected,firewall_log.txt,49665,29


In [27]:
df[df['id_scenario']==29].head()

,id_scenario,ID аномалии,Аномалия,Уверенность,ID проблемы,Файл с проблемой,№ строки,Строка из лога
133,29,220,Network anomaly detected,0.595,NaN,None,NaN,None


In [28]:
anomalies_problems[anomalies_problems['ID аномалии'] == 220]

,ID аномалии,Аномалия,ID проблемы,Проблема
271,220,Network inaccessible,28,Router routing table misconfiguration


### Сценарий 30

In [29]:
all_tests[all_tests['id_scenario'] == 30].head()

,datetime,level,source,text,filename,line_number,id_scenario
123,2025-01-09T22:50:25,WARNING,network,Network routing anomaly detected,app_server1_log.txt,49826,30
124,2025-01-09T22:22:45,WARNING,network,Routing protocol issue detected,backup_server_log.txt,48166,30
125,2025-01-09T22:41:31,WARNING,network,VM network access failure,backup_server_log.txt,49292,30
126,2025-01-09T22:14:49,ERROR,network,Virtual network misconfiguration,router_log.txt,47690,30
127,2025-01-09T22:50:19,WARNING,network,Virtual network connectivity error,switch2_log.txt,49820,30


In [30]:
df[df['id_scenario']==30].head()

,id_scenario,ID аномалии,Аномалия,Уверенность,ID проблемы,Файл с проблемой,№ строки,Строка из лога
134,30,219,Network routing anomaly detected,0.630,NaN,None,NaN,None
135,30,219,Routing protocol issue detected,0.673,NaN,None,NaN,None
136,30,367,VM network access failure,1.000,46.0,router_log.txt,47690.0,2025-01-09T22:14:49 ERROR network: Virtual net...
137,30,363,Virtual network connectivity error,1.000,46.0,router_log.txt,47690.0,2025-01-09T22:14:49 ERROR network: Virtual net...


In [31]:
anomalies_problems[anomalies_problems['ID аномалии'] == 219]

,ID аномалии,Аномалия,ID проблемы,Проблема
270,219,Routing error logged,28,Router routing table misconfiguration
